In [69]:
import pandas as pd
sms = pd.read_csv('sms_spam.csv')
sms.head()

print(sms.dtypes)

type    object
text    object
dtype: object


In [70]:
import re
def pre_process(info):
    #lowercase
    info = info.lower()
    #remove tags
    info=re.sub("</?.*?>"," <> ",info)
    # remove special characters and digits
    info=re.sub("(\\d|\\W)+"," ",info)
    return info

sms['info'] = sms['type'] + sms['text']
sms['info'] = sms['info'].apply(lambda x:pre_process(x))

sms['text'][:]


0       Hope you are having a good week. Just checking in
1                                 K..give back my thanks.
2             Am also doing in cbe only. But have to pay.
3       complimentary 4 STAR Ibiza Holiday or £10,000 ...
4       okmail: Dear Dave this is your final notice to...
                              ...                        
5554    You are a great role model. You are giving so ...
5555    Awesome, I remember the last time we got someb...
5556    If you don't, your prize will go to another cu...
5557    SMS. ac JSco: Energy is high, but u may not kn...
5558                      Shall call now dear having food
Name: text, Length: 5559, dtype: object

text l


In [131]:
from nltk import stem
from nltk.corpus import stopwords
import nltk



nltk.download('stopwords')
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))
def alternative_review_messages(msg):    
    # converting messages to lowercase    
    msg = msg.lower()    
    # removing stopwords   
    msg = [word for word in msg.split() if word not in stopwords]    
    # using a stemmer    
    msg = " ".join([stemmer.stem(word) for word in msg])    
    return msg

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alonso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
word_count_vector.shape

(5559, 8530)

In [121]:

cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape



(5559, 8530)

In [101]:
list(cv.vocabulary_.keys())[:]

['hope',
 'good',
 'week',
 'checking',
 'give',
 'back',
 'thanks',
 'also',
 'cbe',
 'pay',
 'complimentary',
 'star',
 'ibiza',
 'holiday',
 '10',
 '000',
 'cash',
 'needs',
 'urgent',
 'collection',
 '09066364349',
 'landline',
 'lose',
 'box434sk38wp150ppm18',
 'okmail',
 'dear',
 'dave',
 'final',
 'notice',
 'collect',
 'tenerife',
 '5000',
 'award',
 'call',
 '09061743806',
 'tcs',
 'sae',
 'box326',
 'cw25wx',
 '150ppm',
 'aiya',
 'discuss',
 'later',
 'lar',
 'pick',
 'much',
 'buzy',
 'please',
 'ask',
 'mummy',
 'father',
 'marvel',
 'mobile',
 'play',
 'official',
 'ultimate',
 'spider',
 'man',
 'game',
 '50',
 'ur',
 'right',
 'text',
 '83338',
 'send',
 'free',
 '8ball',
 'wallpaper',
 'fyi',
 'usf',
 'swing',
 'room',
 'whenever',
 'sure',
 'thing',
 'big',
 'hockey',
 'elections',
 'go',
 'longer',
 'hour',
 'though',
 'anything',
 'lor',
 'march',
 'ending',
 'ready',
 'problem',
 'capital',
 'never',
 'complete',
 'far',
 'work',
 'ladies',
 'hmm',
 'well',
 'night'

In [102]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [103]:
df_test = pd.read_csv("sms_spam.csv")
df_test['text'] = df_test['type'] + df_test['text']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()

In [104]:
tfidf_transformer.idf_

array([7.22545811, 6.29114888, 8.93020621, ..., 8.93020621, 8.93020621,
       8.93020621])

In [105]:
# get test docs into a list
docs_test=sms['text'].tolist()

In [106]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [107]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,1000)

In [108]:
print("\n=====SMS=====")
print(docs_test[0])

for k in keywords:
    print(k, keywords[k])


=====SMS=====
Hope you are having a good week. Just checking in
checking 0.662
week 0.455
hope 0.454
good 0.386


In [137]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_test[idx])

    for k in keywords:
        print(k,keywords[k])

In [110]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/alonso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [140]:

msg = ""
for i in range(1000):
    message = docs_test[i]
    msg = msg + message
    continue
    
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(msg)
    


['Hope',
 'you',
 'are',
 'having',
 'a',
 'good',
 'week',
 'Just',
 'checking',
 'inK',
 'give',
 'back',
 'my',
 'thanks',
 'Am',
 'also',
 'doing',
 'in',
 'cbe',
 'only',
 'But',
 'have',
 'to',
 'pay',
 'complimentary',
 '4',
 'STAR',
 'Ibiza',
 'Holiday',
 'or',
 '10',
 '000',
 'cash',
 'needs',
 'your',
 'URGENT',
 'collection',
 '09066364349',
 'NOW',
 'from',
 'Landline',
 'not',
 'to',
 'lose',
 'out',
 'Box434SK38WP150PPM18',
 'okmail',
 'Dear',
 'Dave',
 'this',
 'is',
 'your',
 'final',
 'notice',
 'to',
 'collect',
 'your',
 '4',
 'Tenerife',
 'Holiday',
 'or',
 '5000',
 'CASH',
 'award',
 'Call',
 '09061743806',
 'from',
 'landline',
 'TCs',
 'SAE',
 'Box326',
 'CW25WX',
 '150ppmAiya',
 'we',
 'discuss',
 'later',
 'lar',
 'Pick',
 'u',
 'up',
 'at',
 '4',
 'is',
 'it',
 'Are',
 'you',
 'this',
 'much',
 'buzyPlease',
 'ask',
 'mummy',
 'to',
 'call',
 'fatherMarvel',
 'Mobile',
 'Play',
 'the',
 'official',
 'Ultimate',
 'Spider',
 'man',
 'game',
 '4',
 '50',
 'on',
 

In [112]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

token = word_tokenize(msg)

fdist = FreqDist(token)
fdist
msg = fdist.most_common(20)

In [113]:
print(msg)

[('.', 730), ('to', 399), (',', 356), ('you', 312), ('I', 281), ('?', 261), ('the', 258), ('!', 257), ('a', 226), ('...', 202), ('in', 167), ('i', 164), ('and', 149), ('is', 146), ('u', 134), ('me', 133), (':', 119), ('for', 119), ('of', 110), ('my', 106)]
